In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import f_regression, SelectKBest, RFE
from scipy import stats
import math
from wrangle_zillow import acquire_zillow
from wrangle_zillow import missing_col_values
from wrangle_zillow import missing_row_values
from wrangle_zillow import single_unit_properties
from wrangle_zillow import data_prep
from wrangle_zillow import split_data
from wrangle_zillow import remove_outliers
from wrangle_zillow import split_by_region
from wrangle_zillow import lr_model_prep
from wrangle_zillow import cluster_model_prep


from sklearn.linear_model import TweedieRegressor

In [ ]:
df = acquire_zillow()
df = single_unit_properties(df)
remove_cols = ['buildingqualitytypeid', 'heatingorsystemtypeid', 'propertyzoningdesc', 'heatingorsystemdesc']
df = data_prep(df, cols_to_remove=remove_cols, prop_required_column=.5, prop_required_row=.75)
train, validate, test = split_data(df)
out_columns = ['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet',]
train = remove_outliers(train, 1.5, out_columns)

In [ ]:
sns.scatterplot(x="calculatedfinishedsquarefeet", y="logerror", data=train)
plt.xlabel("home sqr foot")
plt.ylabel("log error")
plt.show()

In [ ]:
#assign an alpha value
alpha = 0.05
#assign the x and y variables from the train dataset
x = train.calculatedfinishedsquarefeet
y = train.logerror
#run a pearson r correlation test
corr, p = stats.pearsonr(x, y)
#output the results of the test
print(f'Correlation Coefficient = {corr} P-Value = {p}')
print('-----------------------------------')
if p < alpha:
    print('We reject the null hypothesis')
else:
    print('We do not reject the null hypothesis')

In [ ]:
sns.barplot(x="bathroomcnt", y="logerror", data=train)
plt.xlabel("baths")
plt.ylabel("log error")
plt.show()

In [ ]:
#assign an alpha value
alpha = 0.05
#assign the x and y variables from the train dataset
x = train.bathroomcnt
y = train.logerror
#run a pearson r correlation test
corr, p = stats.pearsonr(x, y)
#output the results of the test
print(f'Correlation Coefficient = {corr} P-Value = {p}')
print('-----------------------------------')
if p < alpha:
    print('We reject the null hypothesis')
else:
    print('We do not reject the null hypothesis')

In [ ]:
sns.barplot(x="bedroomcnt", y="logerror", data=train)
plt.xlabel("beds")
plt.ylabel("log error")
plt.show()

In [ ]:
#assign an alpha value
alpha = 0.05
#assign the x and y variables from the train dataset
x = train.bedroomcnt
y = train.logerror
#run a pearson r correlation test
corr, p = stats.pearsonr(x, y)
#output the results of the test
print(f'Correlation Coefficient = {corr} P-Value = {p}')
print('-----------------------------------')
if p < alpha:
    print('We reject the null hypothesis')
else:
    print('We do not reject the null hypothesis')

In [ ]:
sns.scatterplot(x="latitude", y="logerror", data=train)
plt.xlabel("latitude")
plt.ylabel("log error")
plt.show()

In [ ]:
#assign an alpha value
alpha = 0.05
#assign the x and y variables from the train dataset
x = train.latitude
y = train.logerror
#run a pearson r correlation test
corr, p = stats.pearsonr(x, y)
#output the results of the test
print(f'Correlation Coefficient = {corr} P-Value = {p}')
print('-----------------------------------')
if p < alpha:
    print('We reject the null hypothesis')
else:
    print('We do not reject the null hypothesis')

In [ ]:
sns.scatterplot(x="longitude", y="logerror", data=train)
plt.xlabel("longitude")
plt.ylabel("log error")
plt.show()

In [ ]:
#assign an alpha value
alpha = 0.05
#assign the x and y variables from the train dataset
x = train.longitude
y = train.logerror
#run a pearson r correlation test
corr, p = stats.pearsonr(x, y)
#output the results of the test
print(f'Correlation Coefficient = {corr} P-Value = {p}')
print('-----------------------------------')
if p < alpha:
    print('We reject the null hypothesis')
else:
    print('We do not reject the null hypothesis')

In [ ]:
df = acquire_zillow()
df = single_unit_properties(df)
remove_cols = ['buildingqualitytypeid', 'heatingorsystemtypeid', 'propertyzoningdesc', 'heatingorsystemdesc']
df = data_prep(df, cols_to_remove=remove_cols, prop_required_column=.5, prop_required_row=.75)
df1, df2, df3 = split_by_region(df)
df1['dist_lat'] = df1.latitude - 34012355
df1['dist_long'] = df1.longitude - (-118498665)
df2['dist_lat'] = df2.latitude - 33640954
df2['dist_long'] = df2.longitude - (-117978893)
df3['dist_lat'] = df3.latitude - 34267111
df3['dist_long'] = df3.longitude - (-119278788)

In [ ]:
train1, validate1, test1 = split_data(df1)
train2, validate2, test2 = split_data(df2)
train3, validate3, test3 = split_data(df3)

out_cols = ['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet',]
train1 = remove_outliers(train1, 1.5, out_cols)
train2 = remove_outliers(train2, 1.5, out_cols)
train3 = remove_outliers(train3, 1.5, out_cols)

cluster_df1_train = cluster_model_prep(train1)
cluster_df1_val = cluster_model_prep(validate1)
cluster_df1_test = cluster_model_prep(test1)

cluster_df2_train = cluster_model_prep(train2)
cluster_df2_val = cluster_model_prep(validate2)
cluster_df2_test = cluster_model_prep(test2)

cluster_df3_train = cluster_model_prep(train3)
cluster_df3_val = cluster_model_prep(validate3)
cluster_df3_test = cluster_model_prep(test3)

Cluster 1

In [ ]:
#df1
x = cluster_df1_train[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet']]
kmeans = KMeans(n_clusters=3, random_state=123)
kmeans.fit(x)
train1['bedbathsqft_cluster'] = kmeans.predict(x)

x = cluster_df1_val[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet']]
kmeans = KMeans(n_clusters=3, random_state=123)
kmeans.fit(x)
validate1['bedbathsqft_cluster'] = kmeans.predict(x)

x = cluster_df1_test[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet']]
kmeans = KMeans(n_clusters=3, random_state=123)
kmeans.fit(x)
test1['bedbathsqft_cluster'] = kmeans.predict(x)



#df2
x = cluster_df2_train[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet']]
kmeans = KMeans(n_clusters=3, random_state=123)
kmeans.fit(x)
train2['bedbathsqft_cluster'] = kmeans.predict(x)

x = cluster_df2_val[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet']]
kmeans = KMeans(n_clusters=3, random_state=123)
kmeans.fit(x)
validate2['bedbathsqft_cluster'] = kmeans.predict(x)

x = cluster_df2_test[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet']]
kmeans = KMeans(n_clusters=3, random_state=123)
kmeans.fit(x)
test2['bedbathsqft_cluster'] = kmeans.predict(x)


#df3
x = cluster_df3_train[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet']]
kmeans = KMeans(n_clusters=3, random_state=123)
kmeans.fit(x)
train3['bedbathsqft_cluster'] = kmeans.predict(x)

x = cluster_df3_val[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet']]
kmeans = KMeans(n_clusters=3, random_state=123)
kmeans.fit(x)
validate3['bedbathsqft_cluster'] = kmeans.predict(x)

x = cluster_df3_test[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet']]
kmeans = KMeans(n_clusters=3, random_state=123)
kmeans.fit(x)
test3['bedbathsqft_cluster'] = kmeans.predict(x)






In [ ]:
sns.relplot(data=train1, y='logerror', x='calculatedfinishedsquarefeet', hue='bedbathsqft_cluster')

sns.relplot(data=train2, y='logerror', x='calculatedfinishedsquarefeet', hue='bedbathsqft_cluster')

sns.relplot(data=train3, y='logerror', x='calculatedfinishedsquarefeet', hue='bedbathsqft_cluster')

plt.show()

In [ ]:
clust_1 = train1[train1['bedbathsqft_cluster']== 0]
clust_2 = train1[train1['bedbathsqft_cluster']== 1]
clust_3 = train1[train1['bedbathsqft_cluster']== 2]
print('df1 log error means:')
print(clust_1.logerror.mean(), clust_2.logerror.mean(), clust_3.logerror.mean())

clust_4 = train2[train2['bedbathsqft_cluster']== 0]
clust_5 = train2[train2['bedbathsqft_cluster']== 1]
clust_6 = train2[train2['bedbathsqft_cluster']== 2]
print('df2 log error means:')
print(clust_4.logerror.mean(), clust_5.logerror.mean(), clust_6.logerror.mean())

clust_7 = train3[train3['bedbathsqft_cluster']== 0]
clust_8 = train3[train3['bedbathsqft_cluster']== 1]
clust_9 = train3[train3['bedbathsqft_cluster']== 2]
print('df3 log error means:')
print(clust_7.logerror.mean(), clust_8.logerror.mean(), clust_9.logerror.mean())


Cluster 2

In [ ]:
#df1
x = cluster_df1_train[['latitude', 'longitude']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
train1['latlong_cluster'] = kmeans.predict(x)

x = cluster_df1_val[['latitude', 'longitude']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
validate1['latlong_cluster'] = kmeans.predict(x)

x = cluster_df1_test[['latitude', 'longitude']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
test1['latlong_cluster'] = kmeans.predict(x)


#df2
x = cluster_df2_train[['latitude', 'longitude']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
train2['latlong_cluster'] = kmeans.predict(x)

x = cluster_df2_val[['latitude', 'longitude']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
validate2['latlong_cluster'] = kmeans.predict(x)

x = cluster_df2_test[['latitude', 'longitude']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
test2['latlong_cluster'] = kmeans.predict(x)


#df3
x = cluster_df3_train[['latitude', 'longitude']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
train3['latlong_cluster'] = kmeans.predict(x)

x = cluster_df3_val[['latitude', 'longitude']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
validate3['latlong_cluster'] = kmeans.predict(x)

x = cluster_df3_test[['latitude', 'longitude']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
test3['latlong_cluster'] = kmeans.predict(x)

In [ ]:
sns.relplot(data=train1, y='logerror', x='latitude', hue='latlong_cluster')
sns.relplot(data=train1, y='logerror', x='longitude', hue='latlong_cluster')

sns.relplot(data=train2, y='logerror', x='latitude', hue='latlong_cluster')
sns.relplot(data=train2, y='logerror', x='longitude', hue='latlong_cluster')

sns.relplot(data=train3, y='logerror', x='latitude', hue='latlong_cluster')
sns.relplot(data=train3, y='logerror', x='longitude', hue='latlong_cluster')

plt.show()



In [ ]:
clust_1 = train1[train1['latlong_cluster']== 0]
clust_2 = train1[train1['latlong_cluster']== 1]
clust_3 = train1[train1['latlong_cluster']== 2]
clust_4 = train1[train1['latlong_cluster']== 3]
print('df1 log error means:')
print(clust_1.logerror.mean(), clust_2.logerror.mean(), clust_3.logerror.mean(), clust_4.logerror.mean())

clust_5 = train2[train2['latlong_cluster']== 0]
clust_6 = train2[train2['latlong_cluster']== 1]
clust_7 = train2[train2['latlong_cluster']== 2]
clust_8 = train2[train2['latlong_cluster']== 3]
print('df2 log error means:')
print(clust_5.logerror.mean(), clust_6.logerror.mean(), clust_7.logerror.mean(), clust_8.logerror.mean())

clust_9 = train3[train3['latlong_cluster']== 0]
clust_10 = train3[train3['latlong_cluster']== 1]
clust_11 = train3[train3['latlong_cluster']== 2]
clust_12 = train3[train3['latlong_cluster']== 3]
print('df3 log error means:')
print(clust_9.logerror.mean(), clust_10.logerror.mean(), clust_11.logerror.mean(), clust_12.logerror.mean())


cluster 3

In [ ]:
#df1
x = cluster_df1_train[['dist_lat', 'dist_long']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
train1['dist_cluster'] = kmeans.predict(x)

x = cluster_df1_val[['dist_lat', 'dist_long']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
validate1['dist_cluster'] = kmeans.predict(x)

x = cluster_df1_test[['dist_lat', 'dist_long']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
test1['dist_cluster'] = kmeans.predict(x)

#df2
x = cluster_df2_train[['dist_lat', 'dist_long']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
train2['dist_cluster'] = kmeans.predict(x)

x = cluster_df2_val[['dist_lat', 'dist_long']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
validate2['dist_cluster'] = kmeans.predict(x)

x = cluster_df2_test[['dist_lat', 'dist_long']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
test2['dist_cluster'] = kmeans.predict(x)

#df3
x = cluster_df3_train[['dist_lat', 'dist_long']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
train3['dist_cluster'] = kmeans.predict(x)

x = cluster_df3_val[['dist_lat', 'dist_long']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
validate3['dist_cluster'] = kmeans.predict(x)

x = cluster_df3_test[['dist_lat', 'dist_long']]
kmeans = KMeans(n_clusters=4, random_state=123)
kmeans.fit(x)
test3['dist_cluster'] = kmeans.predict(x)


In [ ]:
sns.relplot(data=train1, y='logerror', x='dist_lat', hue='dist_cluster')
sns.relplot(data=train1, y='logerror', x='dist_long', hue='dist_cluster')

plt.show()

Prep for modeling

In [ ]:
lrmodel_train1_df = lr_model_prep(cluster_df1_train, train1)
lrmodel_val1_df = lr_model_prep(cluster_df1_val, validate1)
lrmodel_test1_df = lr_model_prep(cluster_df1_test, test1)

lrmodel_train2_df = lr_model_prep(cluster_df2_train, train2)
lrmodel_val2_df = lr_model_prep(cluster_df2_val, validate2)
lrmodel_test2_df = lr_model_prep(cluster_df2_test, test2)

lrmodel_train3_df = lr_model_prep(cluster_df3_train, train3)
lrmodel_val3_df = lr_model_prep(cluster_df3_val, validate3)
lrmodel_test3_df = lr_model_prep(cluster_df3_test, test3)


Linear Regression modeling

Model 1

In [ ]:
x_train_features = lrmodel_train1_df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet']]
y_train_features = train1[['logerror']]
x_validate_features = lrmodel_val1_df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet']]
y_validate_features = validate1[['logerror']]

m1d1_predictions = pd.DataFrame({'actual': validate1.logerror})
#add a baseline column to the predictions dataframe with median home price
m1d1_predictions['baseline'] = validate1.logerror.mean()
#create the polynomial model object
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
#fit the polynomial model object with the scaled x_train variables
poly.fit(x_train_features)
#create a dataframe with the results of fitting the polynomial object
x_train_poly = pd.DataFrame(
    poly.transform(x_train_features),
    columns=poly.get_feature_names(x_train_features.columns),
    index=x_train_features.index,
)
#create a linear regression model object
lm = LinearRegression()
#fit the linear regression model with the above created polynomial dataframe
lm.fit(x_train_poly, y_train_features)
#make predictions on the scaled validate data
x_validate_poly = poly.transform(x_validate_features)
#create a column in the predictions dataframe with the resulting model predictions
m1d1_predictions['polynomial only interaction'] = lm.predict(x_validate_poly)
#calculate the mean squared error for the baseline and model predictions against the actual home prices
def calculate_mse_1(y_predicted):
    return mean_squared_error(m1d1_predictions.actual, y_predicted)





x_train_features = lrmodel_train2_df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet']]
y_train_features = train2[['logerror']]
x_validate_features = lrmodel_val2_df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet']]
y_validate_features = validate2[['logerror']]

m1d2_predictions = pd.DataFrame({'actual': validate2.logerror})
#add a baseline column to the predictions dataframe with median home price
m1d2_predictions['baseline'] = validate2.logerror.mean()
#create the polynomial model object
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
#fit the polynomial model object with the scaled x_train variables
poly.fit(x_train_features)
#create a dataframe with the results of fitting the polynomial object
x_train_poly = pd.DataFrame(
    poly.transform(x_train_features),
    columns=poly.get_feature_names(x_train_features.columns),
    index=x_train_features.index,
)
#create a linear regression model object
lm = LinearRegression()
#fit the linear regression model with the above created polynomial dataframe
lm.fit(x_train_poly, y_train_features)
#make predictions on the scaled validate data
x_validate_poly = poly.transform(x_validate_features)
#create a column in the predictions dataframe with the resulting model predictions
m1d2_predictions['polynomial only interaction'] = lm.predict(x_validate_poly)
#calculate the mean squared error for the baseline and model predictions against the actual home prices
def calculate_mse_2(y_predicted):
    return mean_squared_error(m1d2_predictions.actual, y_predicted)






x_train_features = lrmodel_train3_df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet']]
y_train_features = train3[['logerror']]
x_validate_features = lrmodel_val3_df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet']]
y_validate_features = validate3[['logerror']]

m1d3_predictions = pd.DataFrame({'actual': validate3.logerror})
#add a baseline column to the predictions dataframe with median home price
m1d3_predictions['baseline'] = validate3.logerror.mean()
#create the polynomial model object
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
#fit the polynomial model object with the scaled x_train variables
poly.fit(x_train_features)
#create a dataframe with the results of fitting the polynomial object
x_train_poly = pd.DataFrame(
    poly.transform(x_train_features),
    columns=poly.get_feature_names(x_train_features.columns),
    index=x_train_features.index,
)
#create a linear regression model object
lm = LinearRegression()
#fit the linear regression model with the above created polynomial dataframe
lm.fit(x_train_poly, y_train_features)
#make predictions on the scaled validate data
x_validate_poly = poly.transform(x_validate_features)
#create a column in the predictions dataframe with the resulting model predictions
m1d3_predictions['polynomial only interaction'] = lm.predict(x_validate_poly)
#calculate the mean squared error for the baseline and model predictions against the actual home prices
def calculate_mse_3(y_predicted):
    return mean_squared_error(m1d3_predictions.actual, y_predicted)




In [ ]:
print('Model 1 with the Los Angeles County, CA dataset:')
print('--------------------------------------------------')
print(m1d1_predictions.apply(calculate_mse_1).apply(math.sqrt))
print('==================================================')
print('')
print('Model 1 with the Orange County, CA dataset:')
print('--------------------------------------------------')
print(m1d2_predictions.apply(calculate_mse_2).apply(math.sqrt))
print('==================================================')
print('')
print('Model 1 with the Ventura County, CA dataset:')
print('--------------------------------------------------')
print(m1d3_predictions.apply(calculate_mse_3).apply(math.sqrt))
print('==================================================')

Model 2

In [ ]:
x_train_features = lrmodel_train1_df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'latitude', 'longitude']]
y_train_features = train1[['logerror']]
x_validate_features = lrmodel_val1_df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'latitude', 'longitude']]
y_validate_features = validate1[['logerror']]

m2d1_predictions = pd.DataFrame({'actual': validate1.logerror})
#add a baseline column to the predictions dataframe with median home price
m2d1_predictions['baseline'] = validate1.logerror.mean()
#create the polynomial model object
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
#fit the polynomial model object with the scaled x_train variables
poly.fit(x_train_features)
#create a dataframe with the results of fitting the polynomial object
x_train_poly = pd.DataFrame(
    poly.transform(x_train_features),
    columns=poly.get_feature_names(x_train_features.columns),
    index=x_train_features.index,
)
#create a linear regression model object
lm = LinearRegression()
#fit the linear regression model with the above created polynomial dataframe
lm.fit(x_train_poly, y_train_features)
#make predictions on the scaled validate data
x_validate_poly = poly.transform(x_validate_features)
#create a column in the predictions dataframe with the resulting model predictions
m2d1_predictions['polynomial only interaction'] = lm.predict(x_validate_poly)
#calculate the mean squared error for the baseline and model predictions against the actual home prices
def calculate_mse_4(y_predicted):
    return mean_squared_error(m2d1_predictions.actual, y_predicted)




x_train_features = lrmodel_train2_df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'latitude', 'longitude']]
y_train_features = train2[['logerror']]
x_validate_features = lrmodel_val2_df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'latitude', 'longitude']]
y_validate_features = validate2[['logerror']]

m2d2_predictions = pd.DataFrame({'actual': validate2.logerror})
#add a baseline column to the predictions dataframe with median home price
m2d2_predictions['baseline'] = validate2.logerror.mean()
#create the polynomial model object
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
#fit the polynomial model object with the scaled x_train variables
poly.fit(x_train_features)
#create a dataframe with the results of fitting the polynomial object
x_train_poly = pd.DataFrame(
    poly.transform(x_train_features),
    columns=poly.get_feature_names(x_train_features.columns),
    index=x_train_features.index,
)
#create a linear regression model object
lm = LinearRegression()
#fit the linear regression model with the above created polynomial dataframe
lm.fit(x_train_poly, y_train_features)
#make predictions on the scaled validate data
x_validate_poly = poly.transform(x_validate_features)
#create a column in the predictions dataframe with the resulting model predictions
m2d2_predictions['polynomial only interaction'] = lm.predict(x_validate_poly)
#calculate the mean squared error for the baseline and model predictions against the actual home prices
def calculate_mse_5(y_predicted):
    return mean_squared_error(m2d2_predictions.actual, y_predicted)





x_train_features = lrmodel_train3_df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'latitude', 'longitude']]
y_train_features = train3[['logerror']]
x_validate_features = lrmodel_val3_df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'latitude', 'longitude']]
y_validate_features = validate3[['logerror']]

m2d3_predictions = pd.DataFrame({'actual': validate3.logerror})
#add a baseline column to the predictions dataframe with median home price
m2d3_predictions['baseline'] = validate3.logerror.mean()
#create the polynomial model object
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
#fit the polynomial model object with the scaled x_train variables
poly.fit(x_train_features)
#create a dataframe with the results of fitting the polynomial object
x_train_poly = pd.DataFrame(
    poly.transform(x_train_features),
    columns=poly.get_feature_names(x_train_features.columns),
    index=x_train_features.index,
)
#create a linear regression model object
lm = LinearRegression()
#fit the linear regression model with the above created polynomial dataframe
lm.fit(x_train_poly, y_train_features)
#make predictions on the scaled validate data
x_validate_poly = poly.transform(x_validate_features)
#create a column in the predictions dataframe with the resulting model predictions
m2d3_predictions['polynomial only interaction'] = lm.predict(x_validate_poly)
#calculate the mean squared error for the baseline and model predictions against the actual home prices
def calculate_mse_6(y_predicted):
    return mean_squared_error(m2d3_predictions.actual, y_predicted)


In [ ]:
print('Model 2 with the Los Angeles County, CA dataset:')
print('--------------------------------------------------')
print(m2d1_predictions.apply(calculate_mse_4).apply(math.sqrt))
print('==================================================')
print('')
print('Model 2 with the Orange County, CA dataset:')
print('--------------------------------------------------')
print(m2d2_predictions.apply(calculate_mse_5).apply(math.sqrt))
print('==================================================')
print('')
print('Model 2 with the Ventura County, CA dataset:')
print('--------------------------------------------------')
print(m2d3_predictions.apply(calculate_mse_6).apply(math.sqrt))
print('==================================================')

Model 3

In [ ]:
x_train_features = lrmodel_train1_df[['bedbathsqft_cluster_1', 'bedbathsqft_cluster_2', 'latlong_cluster_1', 'latlong_cluster_2',
       'latlong_cluster_3']]
y_train_features = train1[['logerror']]
x_validate_features = lrmodel_val1_df[['bedbathsqft_cluster_1', 'bedbathsqft_cluster_2', 'latlong_cluster_1', 'latlong_cluster_2',
       'latlong_cluster_3']]
y_validate_features = validate1[['logerror']]

m4d1_predictions = pd.DataFrame({'actual': validate1.logerror})
#add a baseline column to the predictions dataframe with median home price
m4d1_predictions['baseline'] = validate1.logerror.mean()
#create the polynomial model object
poly = PolynomialFeatures(degree=5, include_bias=False, interaction_only=True)
#fit the polynomial model object with the scaled x_train variables
poly.fit(x_train_features)
#create a dataframe with the results of fitting the polynomial object
x_train_poly = pd.DataFrame(
    poly.transform(x_train_features),
    columns=poly.get_feature_names(x_train_features.columns),
    index=x_train_features.index,
)
#create a linear regression model object
lm = LinearRegression()
#fit the linear regression model with the above created polynomial dataframe
lm.fit(x_train_poly, y_train_features)
#make predictions on the scaled validate data
x_validate_poly = poly.transform(x_validate_features)
#create a column in the predictions dataframe with the resulting model predictions
m4d1_predictions['polynomial only interaction'] = lm.predict(x_validate_poly)
#calculate the mean squared error for the baseline and model predictions against the actual home prices
def calculate_mse_10(y_predicted):
    return mean_squared_error(m4d1_predictions.actual, y_predicted)
##########################
x_train_features = lrmodel_train2_df[['bedbathsqft_cluster_1', 'bedbathsqft_cluster_2', 'latlong_cluster_1', 'latlong_cluster_2',
       'latlong_cluster_3']]
y_train_features = train2[['logerror']]
x_validate_features = lrmodel_val2_df[['bedbathsqft_cluster_1', 'bedbathsqft_cluster_2', 'latlong_cluster_1', 'latlong_cluster_2',
       'latlong_cluster_3']]
y_validate_features = validate2[['logerror']]

m4d2_predictions = pd.DataFrame({'actual': validate2.logerror})
#add a baseline column to the predictions dataframe with median home price
m4d2_predictions['baseline'] = validate2.logerror.mean()
#create the polynomial model object
poly = PolynomialFeatures(degree=5, include_bias=False, interaction_only=True)
#fit the polynomial model object with the scaled x_train variables
poly.fit(x_train_features)
#create a dataframe with the results of fitting the polynomial object
x_train_poly = pd.DataFrame(
    poly.transform(x_train_features),
    columns=poly.get_feature_names(x_train_features.columns),
    index=x_train_features.index,
)
#create a linear regression model object
lm = LinearRegression()
#fit the linear regression model with the above created polynomial dataframe
lm.fit(x_train_poly, y_train_features)
#make predictions on the scaled validate data
x_validate_poly = poly.transform(x_validate_features)
#create a column in the predictions dataframe with the resulting model predictions
m4d2_predictions['polynomial only interaction'] = lm.predict(x_validate_poly)
#calculate the mean squared error for the baseline and model predictions against the actual home prices
def calculate_mse_11(y_predicted):
    return mean_squared_error(m4d2_predictions.actual, y_predicted)
###########################
x_train_features = lrmodel_train3_df[['bedbathsqft_cluster_1', 'bedbathsqft_cluster_2', 'latlong_cluster_1', 'latlong_cluster_2',
       'latlong_cluster_3']]
y_train_features = train3[['logerror']]
x_validate_features = lrmodel_val3_df[['bedbathsqft_cluster_1', 'bedbathsqft_cluster_2', 'latlong_cluster_1', 'latlong_cluster_2',
       'latlong_cluster_3']]
y_validate_features = validate3[['logerror']]

m4d3_predictions = pd.DataFrame({'actual': validate3.logerror})
#add a baseline column to the predictions dataframe with median home price
m4d3_predictions['baseline'] = validate3.logerror.mean()
#create the polynomial model object
poly = PolynomialFeatures(degree=5, include_bias=False, interaction_only=True)
#fit the polynomial model object with the scaled x_train variables
poly.fit(x_train_features)
#create a dataframe with the results of fitting the polynomial object
x_train_poly = pd.DataFrame(
    poly.transform(x_train_features),
    columns=poly.get_feature_names(x_train_features.columns),
    index=x_train_features.index,
)
#create a linear regression model object
lm = LinearRegression()
#fit the linear regression model with the above created polynomial dataframe
lm.fit(x_train_poly, y_train_features)
#make predictions on the scaled validate data
x_validate_poly = poly.transform(x_validate_features)
#create a column in the predictions dataframe with the resulting model predictions
m4d3_predictions['polynomial only interaction'] = lm.predict(x_validate_poly)
#calculate the mean squared error for the baseline and model predictions against the actual home prices
def calculate_mse_12(y_predicted):
    return mean_squared_error(m4d3_predictions.actual, y_predicted)



In [ ]:
print('Model 4 with the Los Angeles County, CA dataset:')
print('--------------------------------------------------')
print(m4d1_predictions.apply(calculate_mse_10).apply(math.sqrt))
print('==================================================')
print('')
print('Model 4 with the Orange County, CA dataset:')
print('--------------------------------------------------')
print(m4d2_predictions.apply(calculate_mse_11).apply(math.sqrt))
print('==================================================')
print('')
print('Model 4 with the Ventura County, CA dataset:')
print('--------------------------------------------------')
print(m4d3_predictions.apply(calculate_mse_12).apply(math.sqrt))
print('==================================================')

model 4